In [1]:
from annainflux import client
import pandas as pd

In [2]:
rs = client.query("SHOW TAG VALUES FROM dependencies WITH KEY=project")
nodes = pd.DataFrame(rs.get_points())
nodes = nodes.drop(columns=['key'])
nodes = nodes.append({'value': 'qtbase'}, ignore_index=True)

projects = nodes['value'].tolist()
projectCount = len(projects)

In [3]:
rs = client.query("SELECT * FROM dependencies")
links = pd.DataFrame(rs.get_points())
links = links.drop(columns=['branch', 'depHeadSha1', 'time'])
links = links.rename(columns={"project": "source", "dependency": "target", "howFar": "value"})
#order change: i want an obvious order: source->target, value, where sorce is project, target is dependency, value is no of commits
links = links[['source', 'target','value']]
#i also need labelrs
sourceNames = links['source'].tolist()

In [4]:
depMatrix = pd.DataFrame(data=None, index=range(0,projectCount), columns=range(0,projectCount), dtype=None, copy=False)
sources = []
targets = []
for row in links.iterrows():        
    sources.append(projects.index(row[1]['source']))    
    targets.append(projects.index(row[1]['target']))    
    depMatrix.loc[projects.index(row[1]['source']),projects.index(row[1]['target'])] = row[1]['value']


In [5]:
#that is helper function I need it to present values(how many commits project is ahead of its dependency)
def howManyCommits(link):    
    if pd.isna(depMatrix.iloc[link[0],link[1]]):    
        return ""
    else:
        return str(depMatrix.iloc[link[0],link[1]])

In [51]:
#thsi is temorpary i created hard coded color map, will replace it by some sensible function in the future

colorMap = []
labelList=[]
for (i, j) in zip(sources, targets):
    val = depMatrix.iloc[i,j]
    if val == 0:
        colorMap.append('#2ca25f')
        labelList.append('0: up to date')
    if val == 1 :
        colorMap.append('#99d8c9')
        labelList.append('1 commit')
    if val == 4 :
        colorMap.append('#fdbb84')
        labelList.append('4 commits')
    


In [52]:
import math
from numpy import isnan
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file
from bokeh.models import TapTool, OpenURL, ColumnDataSource,  Legend, LegendItem

heatMap = figure(title="Qt modules dependency matrix", 
            tools="tap", toolbar_location="below",                        
            y_range = projects,y_axis_label = 'Qt module',
            x_range = projects, x_axis_label = 'dependency',
            plot_width=800, plot_height=800,)

heatMap.xaxis.major_label_orientation = math.pi/2           

taptool = heatMap.select(type=TapTool)
taptool.callback = OpenURL(url="https://testresults.qt.io/grafana/d/pJyKbjXZz/git?orgId=1&var-project=@y")


#careful here!
#sources: qtprojects, are on Y axis
#targets: dependent modules are on X axis
moduleDependencyData = ColumnDataSource(data=dict(x=targets,y=sourceNames, color= colorMap, label=labelList))
heatMap.circle('x', 'y', color='color', size=22, legend_group='label', source=moduleDependencyData)

#Again careful here - targets (dependencies) are on x axis
# sources qt modules are on y axis,  can be super confusing!
heatMap.text(targets, sourceNames, text=["%s" % howManyCommits((i,j)) for (i, j) in zip(sources, targets)], text_align="left", text_baseline = "middle")

output_file("qtModulesDependcyMap.html", title="qt modules dependency map")
show(heatMap)  # open a browser